In [1]:
from flask import Flask,render_template,url_for,request
import re  
import nltk  
nltk.download('stopwords') 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 
import pandas as pd 
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.externals import joblib
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pathak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dataset = pd.read_csv('review.csv')

C:\Users\pathak\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Summary Table
percent_missing = dataset.isnull().sum()*100/len(dataset)
data_type = dataset.dtypes
summary = pd.DataFrame({"Percent_missing": percent_missing.round(2), 
                               "Datatypes": data_type})
summary

,Percent_missing,Datatypes
id,0.00,object
name,19.50,object
asins,0.01,object
brand,0.00,object
categories,0.00,object
keys,0.00,object
manufacturer,0.00,object
date,0.11,object
dateAdded,30.64,object
dateSeen,0.00,object


In [4]:
# Detailed profiling using pandas profiling
import pandas_profiling
pandas_profiling.ProfileReport(dataset) 

Number of variables,21
Number of observations,34660
Total Missing (%),21.6%
Total size in memory,5.6 MiB
Average record size in memory,168.0 B
Numeric,2
Categorical,15
Boolean,1
Date,0
Text (Unique),1
Rejected,2


In [5]:
# Acccumulating high cardinal, high correlated variables, high missing as per the audit report
col_list_drop=['username','userProvince','doRecommend','userCity','title','sourceURLs','numHelpful','id.1','dateSeen','dateAdded','date','name','didPurchase','asins','keys','manufacturer']

In [6]:
# Drop the accumulated data
dataset=dataset.drop(col_list_drop,axis=1)

In [7]:
# Summary Table
percent_missing = dataset.isnull().sum()
data_type = dataset.dtypes
summary = pd.DataFrame({"Percent_missing": percent_missing.round(2), 
                               "Datatypes": data_type})
summary

,Percent_missing,Datatypes
id,0,object
brand,0,object
categories,0,object
rating,33,float64
text,1,object


In [8]:
dataset = dataset.dropna()
#dataset = drop[dataset["text"].notnull()]
#data.head()

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34626 entries, 0 to 34659
Data columns (total 5 columns):
id            34626 non-null object
brand         34626 non-null object
categories    34626 non-null object
rating        34626 non-null float64
text          34626 non-null object
dtypes: float64(1), object(4)
memory usage: 1.6+ MB


In [10]:
dataset["star"] = dataset["rating"]>=4
dataset["star"] = dataset["star"].replace([True , False] , ["0" , "1"])

In [11]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
import numpy as np
import re
import string
import nltk
clean_re = re.compile('[^a-z]+')
def cleanup(line):
    line = str(line)
    line = line.lower()
    line = clean_re.sub(' ', line).strip()
    return line

In [12]:
dataset["stext"] = dataset["text"].apply(cleanup)

In [13]:
dataset

,id,brand,categories,rating,text,star,stext
0,AVqkIhwDv8e3D1O-lebb,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,This product so far has not disappointed. My c...,0,this product so far has not disappointed my ch...
1,AVqkIhwDv8e3D1O-lebb,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,great for beginner or experienced person. Boug...,0,great for beginner or experienced person bough...
2,AVqkIhwDv8e3D1O-lebb,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,Inexpensive tablet for him to use and learn on...,0,inexpensive tablet for him to use and learn on...
3,AVqkIhwDv8e3D1O-lebb,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",4.0,I've had my Fire HD 8 two weeks now and I love...,0,i ve had my fire hd two weeks now and i love i...
4,AVqkIhwDv8e3D1O-lebb,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,I bought this for my grand daughter when she c...,0,i bought this for my grand daughter when she c...
5,AVqkIhwDv8e3D1O-lebb,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,This amazon fire 8 inch tablet is the perfect ...,0,this amazon fire inch tablet is the perfect si...
6,AVqkIhwDv8e3D1O-lebb,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",4.0,"Great for e-reading on the go, nice and light ...",0,great for e reading on the go nice and light w...
7,AVqkIhwDv8e3D1O-lebb,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,"I gave this as a Christmas gift to my inlaws, ...",0,i gave this as a christmas gift to my inlaws h...
8,AVqkIhwDv8e3D1O-lebb,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,Great as a device to read books. I like that i...,0,great as a device to read books i like that it...
9,AVqkIhwDv8e3D1O-lebb,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,I love ordering books and reading them with th...,0,i love ordering books and reading them with th...


In [14]:
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
#cv = CountVectorizer(max_features = 25000)
tf_fit = tf.fit_transform(dataset["stext"])   
#dataset['tf_fit'] = tf.fit_transform(cv_fit)

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tf_fit, dataset["star"], test_size=0.3, random_state=1)

In [16]:
#logistic regression
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=1)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
y_pred_tr = classifier.predict(x_train)

In [17]:
print('Test accuracy', sum(y_test == y_pred)/len(y_test))
print('Train accuracy', sum(y_train == y_pred_tr)/len(y_train))

Test accuracy 0.9396418944936465
Train accuracy 0.9442198201171714


In [18]:
from sklearn.metrics import classification_report
print("Classification Report(Train)")
print(classification_report(y_train, y_pred_tr))
print("Classification Report(Test)")
print(classification_report(y_test, y_pred))

Classification Report(Train)
             precision    recall  f1-score   support

          0       0.95      1.00      0.97     22607
          1       0.87      0.20      0.33      1631

avg / total       0.94      0.94      0.93     24238

Classification Report(Test)
             precision    recall  f1-score   support

          0       0.94      1.00      0.97      9708
          1       0.69      0.14      0.23       680

avg / total       0.93      0.94      0.92     10388



In [19]:
# Random Forests
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
y_pred_tr = classifier.predict(x_train)

In [20]:
print('Test accuracy', sum(y_test == y_pred)/len(y_test))
print('Train accuracy', sum(y_train == y_pred_tr)/len(y_train))

Test accuracy 0.9354062379668848
Train accuracy 0.9886129218582391


In [21]:
#Naive bayes
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
y_pred = classifier.predict(x_test)
y_pred_tr = classifier.predict(x_train)

In [23]:
print('Test accuracy', sum(y_test == y_pred)/len(y_test))
print('Train accuracy', sum(y_train == y_pred_tr)/len(y_train))

Test accuracy 0.9354062379668848
Train accuracy 0.9886129218582391
